In [4]:
from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
import cgi
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from database_setup import Base, Restaurant, MenuItem

In [12]:
engine = create_engine('sqlite:///restaurantmenu.db')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [36]:
class WebServerHandler(BaseHTTPRequestHandler):

    def do_GET(self):
        if self.path.endswith("/hello"):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            output = ""
            output += "<html><body>Hello!"
            output += "<form method='POST' enctype='multipart/form-data' action='hello/'> \
                    <h2>What would you like me to day?</h2><input name='message' type='text'> \
                        <input type='submit' value='Submit'></form>"
            output += "</body></html>"
            self.wfile.write(output)
            print output
            return        
        if self.path.endswith('/restaurant'):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            output = ""
            output += "<html><body>"
            restaurants=session.query(Restaurant).all()
            for r in restaurants:
                output += "<p>" + r.name
                output += "<p><a href='/hello'>Edit</a>"
                output += "<p><a href='/hello'>Delete</a>"
            output += "<p><a href='/new'>Make New Restaurant</a>"
            output += "</body></html>"
            self.wfile.write(output)
            print output
            return 
        if self.path.endswith('/new'):
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            output = ""
            output += "<html><body><h2>Make New Restaurant</h2>"
            output += "<form method='POST' enctype='multipart/form-data' action='hello/'> \
                    <input name='restaurant' type='text'><input type='submit' value='Submit'></form>"
            output += "</body></html>"
            self.wfile.write(output)
            print output
            return   
        else:
            self.send_error(404, 'File Not Found: %s' % self.path)
    def do_POST(self):
        try:
            self.send_response(301)
            self.end_headers()
            ctype, pdict = cgi.parse_header(
                self.headers.getheader('content-type'))
            if ctype == 'multipart/form-data':
                fields = cgi.parse_multipart(self.rfile, pdict)
                messagecontent = fields.get('restaurant')
            print messagecontent
            
            first = Restaurant(name = messagecontent[0])
            session.add(first)
            session.commit()        
            output = ""
            output += "<html><body>"
            output += "<h2> Added %s </h2>" % messagecontent[0]
            output += "<p><a href='/restaurant'>Restaurant edit</a>"
            output += "</body></html>"
            self.wfile.write(output)
            print output
        except:
            print 'yikes'
            pass

In [37]:
def main():
    try:
        port = 8080
        server = HTTPServer(('', port), WebServerHandler)
        print "Web Server running on port %s" % port
        server.serve_forever()
    except KeyboardInterrupt:
        print " ^C entered, stopping web server...."
        server.socket.close()

if __name__ == '__main__':
    main()

Web Server running on port 8080
<html><body>Hello!<form method='POST' enctype='multipart/form-data' action='hello/'>                     <h2>What would you like me to day?</h2><input name='message' type='text'>                         <input type='submit' value='Submit'></form></body></html>

127.0.0.1 - - [22/Jan/2017 08:08:43] "GET /hello HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2017 08:08:45] "GET /new HTTP/1.1" 200 -



<html><body><h2>Make New Restaurant</h2><form method='POST' enctype='multipart/form-data' action='hello/'>                     <input name='restaurant' type='text'><input type='submit' value='Submit'></form></body></html>
<html><body>Hello!<form method='POST' enctype='multipart/form-data' action='hello/'>                     <h2>What would you like me to day?</h2><input name='message' type='text'>                         <input type='submit' value='Submit'></form></body></html>

127.0.0.1 - - [22/Jan/2017 08:08:50] "GET /hello HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2017 08:08:52] "GET /new HTTP/1.1" 200 -



<html><body><h2>Make New Restaurant</h2><form method='POST' enctype='multipart/form-data' action='hello/'>                     <input name='restaurant' type='text'><input type='submit' value='Submit'></form></body></html>
['Yum']

127.0.0.1 - - [22/Jan/2017 08:08:55] "POST /hello/ HTTP/1.1" 301 -
127.0.0.1 - - [22/Jan/2017 08:08:57] "GET /restaurant HTTP/1.1" 200 -



<html><body><h2> Added Yum </h2><p><a href='/restaurant'>Restaurant edit</a></body></html>
<html><body><p>Pizza<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Pizza Palace<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Pizza Palace<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Urban Burger<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Super Stir Fry<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Panda Garden<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Thyme for That Vegetarian Cuisine <p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Tony's Bistro <p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Andala's<p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Auntie Ann's Diner' <p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>Cocina Y Amor <p><a href='/hello'>Edit</a><p><a href='/hello'>Delete</a><p>State Bird Provisions<p><a href='/hello'>Edit</a><p><a href='/hel